## 1. 字符编码
计算机底层只有`0`和`1`。为了让计算机能处理文本，我们必须定义一套**数字（码）与字符的映射规则**，这就是**字符编码**。

理解字符编码，首先要分清两个核心概念：字符集和编码方式
- **字符集 (Character Set)**：一个字符的集合（比如所有英文字母、常用汉字都能构成一个字符集）
- **编码方式 (Encoding)**：将字符集中的字符**转换（映射）为二进制数据**的具体规则

### 1.1 ASCII
早期为了解决英文文本的处理问题，诞生了 ASCII（美国信息交换标准码）
- **逻辑**：使用**1个字节（8位）** 的后7位，为128个字符（包括英文字母、数字、控制符）分配唯一编号。
- **例子**：字符 `“A”` -> 十进制 `65` -> 二进制 `01000001`。
- **局限**：只能表示拉丁字母等，无法表示中文、日文等其他语言的字符。

### 1.2 Unicode
Unicode 的核心目标是打破不同语言编码的壁垒，为全球几乎所有字符分配一个唯一的、通用的数字编号，这个编号被称为 “码点（Code Point）”。比如 “A” 的码点依然延续了 ASCII 的 65（十六进制表示为 U+0041），而汉字 “牛” 的码点则是 29275（十六进制 U+725B）。
- **局限**：Unicode **只解决了“字符编号”** 的问题，但**没有规定这个编号在计算机中如何存储和传输**。

### 1.3 UTF-8
为了填补 Unicode 在 “存储传输” 上的空白，**Unicode转换格式 (Unicode Transformation Format, UTF)**应运而生，常见的有 UTF-32、UTF-16 和 UTF-8 三种形式，它们的设计逻辑和适用场景各有不同。

| 编码方式 | 存储规则             | 优点                               | 缺点                                     | 示例 `“A”` (U+0041) | 示例 `“牛”` (U+725B) |
| :--- | :--- | :--- | :--- | :--- | :--- |
| **UTF-32** | **固定4字节**        | 读取速度快，无需计算长度           | **极度浪费空间**（所有字符都占4字节）     | 4 bytes            | 4 bytes             |
| **UTF-16** | **2或4字节**（变长） | 历史遗留产物 (Java/Windows 核心)   | “不长不短”，变长处理也麻烦               | 2 bytes            | 2 bytes             |
| **UTF-8**  | **变长 (1-4字节)**   | **空间效率高**，完美兼容ASCII      | 读取前需要解析字节数，速度稍慢           | **1 byte**         | **3 bytes**         |

UTF-8采用**“前缀识别法”**，通过**字节开头的前几位（控制位）** 来判断一个字符由几个字节组成。

**编码模板表**
| 码点范围 (Unicode) | 所需字节数 | 字节 1 (模板) | 字节 2 | 字节 3 | 字节 4 |
| :--- | :---: | :--- | :--- | :--- | :--- |
| `U+0000` ~ `U+007F` (ASCII) | **1** | `0xxxxxxx` | - | - | - |
| `U+0080` ~ `U+07FF` | **2** | `110xxxxx` | `10xxxxxx` | - | - |
| `U+0800` ~ `U+FFFF` (如常用汉字) | **3** | `1110xxxx` | `10xxxxxx` | `10xxxxxx` | - |
| `U+10000` ~ `U+10FFFF` | **4** | `11110xxx` | `10xxxxxx` | `10xxxxxx` | `10xxxxxx` |

**关键规则**：
- **控制位**：第一个字节的开头 `1` 的个数表示总字节数（如 `1110` 开头表示共3字节）。
- **延续位**：后续字节都以 `10` 开头，表示它们是“从属字节”。


#### 1.3.1 实例：汉字“牛”的UTF-8编码过程
1.  **获取码点 (Code Point)**:


In [1]:
print(ord("牛"))   # 输出：29275 (十进制)
print(hex(29275))  # 输出：0x725b (十六进制，即 U+725B)

29275
0x725b


2.  **确定模板**:
    - `0x725B` 在 `U+0800` ~ `U+FFFF` 范围内，对应 **3字节模板**：`1110xxxx 10xxxxxx 10xxxxxx`
3.  **码点转二进制并填充**:
    - 将码点 `29275` 转为二进制：`0111 0010 0101 1011` (共16位，Unicode常用16位表示BMP内的字符)
    - 将这个16位的二进制数，**从右到左**依次填入模板的 `x` 位（模板中除去控制位后剩下的位数）：
        ```
        模板:      1110 xxxx 10 xxxxxx 10 xxxxxx
        码点二进制:      0111    001001    011011
        填充后:     1110 0111 10 001001 10 011011
        ```
4.  **得到最终字节**:
    - 字节1: `11100111` -> `0xE7` (十进制 231)
    - 字节2: `10001001` -> `0x89` (十进制 137)
    - 字节3: `10011011` -> `0x9B` (十进制 155)

In [2]:
print("牛".encode('utf-8'))  # 输出：b'\xe7\x89\x9b'，与计算结果一致！

b'\xe7\x89\x9b'


## 2. BPE算法
- [可视化分词结果](https://tiktokenizer.vercel.app/)

Transformer模型的自注意力机制计算复杂度是 \(O(N^2)\)（\(N\) 是序列长度），如果直接把文本转成UTF-8字节喂给模型，会导致两个致命问题：
1. **序列过长**：比如单词“Transformer”有11个字节，序列长度直接拉满，模型计算速度慢、显存占用高；
2. **语义碎片化**：单个字节没有语义（比如“中”的UTF-8是3个字节），模型难以学习到完整的语义单元。

BPE的本质就是**“数据压缩”**：通过合并语料中高频出现的相邻字节对，生成更大的语义单元（Token），在“词典大小”和“序列长度”之间找到最优平衡——既不会让词典过大（增加模型参数），也不会让序列过长（增加计算成本）。
### 2.1 贪婪的高频合并算法
BPE（Byte Pair Encoding）的核心逻辑非常简单，总结为一句话：**从最基础的字节（0-255）开始，反复找到并合并语料中出现频率最高的相邻字符对，直到词表大小达到预设值**。

假设语料只有3个单词及频次：`low(5次)`、`lower(2次)`、`newest(6次)`。
1. 初始状态：所有字符都是独立字节（`l`/`o`/`w`/`e`/`r`/`n`/`s`/`t`）；
2. 第一轮合并：统计所有相邻对，`(e,w)` 出现8次（2+6），是最高频的，合并为`ew`，词表新增`ew`；
3. 第二轮合并：统计更新后的对，`(ew,e)` 出现6次，合并为`ewe`，词表新增`ewe`；
4. 重复此过程，直到词表达到目标大小（比如1000）。
### 2.2 完整实现
#### 2.2.1 初始化基础词表
BPE 的第一步，就是把任何文本先拆成最小的字节单元，再用初始词表（0-255）给每个字节分配 ID。


In [31]:
vocab = {i: bytes([i]) for i in range(256)}
# 创建一个名为 vocab 的字典，键是0-255的整数（对应Token ID），值是每个整数对应的字节
# 因为bytes强制类型转换会把整数i转换成单个字节
# vocab[65] 的值是 b'A'（因为 ASCII 中 65 对应大写字母 A）；
# vocab[228] 的值是 b'\xe4'（UTF-8 中中文 “中” 的第一个字节就是 228，即 0xe4）；
# vocab[0] 的值是 b'\x00'（空字节），vocab[255] 的值是 b'\xff'。

#### 2.2.2 隔离特殊Token
特殊Token（如`<endoftext>`、`[PAD]`）是“功能型Token”，必须保证原子性（不被拆分、不与普通文本合并），所以第一步要把它们从训练语料中隔离：

In [ ]:
import re
# 先定义特殊Token列表（示例）
special_tokens = ["<endoftext>", "[PAD]"]
# 待处理的语料文本
text = "Hello<endoftext>World[PAD]Python"

special_regex = "|".join(re.escape(t) for t in special_tokens)
# re.escape(t)：对特殊 Token 里的特殊字符（比如<、]）做转义，避免正则识别错误（比如<在正则里有特殊含义，转义后才会当成普通字符）；
# "|".join(...)：把所有特殊 Token 用|连接，形成 “或” 的正则规则。
parts = re.split(f"({special_regex})", text)
# 正则里加括号后，能先把文本拆分成「普通文本 + 特殊 Token」的混合列表
print(parts)
train_segments = [part for part in parts if part not in special_tokens]
print(train_segments)


['Hello', '<endoftext>', 'World', '[PAD]', 'Python']
['Hello', 'World', 'Python']


比如语料是`Hello<endoftext>World`，会被切分为`['Hello', '<endoftext>', 'World']`，只取`['Hello', 'World']`用于训练。
#### 2.2.3 预分词（Pre-tokenization）
预分词的核心作用是把文本拆成 “**语义独立的小单元**”（比如单词、标点、数字、中文词），让后续 BPE 只在这些小单元内部合并字节，而不会跨单元合并（比如不会把 “don't” 里的t和后面 “know” 里的k合并），避免产生无意义的 Token，破坏语义。
| 正则片段 | 含义 | 例子 |
|----------|------|------|
| `'s\|'t\|'re\|'ve\|'m\|'ll\|'d` | 匹配英文中常见的缩写后缀（独立成单元） | `don't` 会先匹配`'t`，剩下的`don`后续匹配 |
| `[\p{L}]+` | 匹配任意语言的字母/文字（`\p{L}`是Unicode属性，代表所有语言的字符，包括中文、英文、日文等），`+`表示至少1个 | `I`、`don`、`know`、`你好` |
| `[\p{N}]+` | 匹配数字（`\p{N}`是Unicode属性，代表所有数字） | `123` |
| `[^\s\p{L}\p{N}]+?` | 匹配非空白、非文字、非数字的字符（即标点、符号），`+?`是非贪婪匹配，保证每个符号尽可能独立 | `.`、`!` |
| `re.UNICODE` | 启用Unicode匹配（让`\p{L}`/`\p{N}`生效） | 能识别中文、日文等非英文文本 |

In [11]:
import regex
# GPT-2经典的预分词正则（带Unicode支持）
gpt2_pattern = regex.compile(r"""'s|'t|'re|'ve|'m|'ll|'d|[\p{L}]+|[\p{N}]+|[^\s\p{L}\p{N}]+?""", re.UNICODE)

text = "I don't know. 你好123!"
words = gpt2_pattern.findall(text)
print(words) 
# 1. 先扫描`I` → 匹配`[\p{L}]+` → 提取`I`；
# 2. 扫描`don't` → 先匹配`'t`（缩写规则），剩下的`don`匹配`[\p{L}]+` → 提取`don`、`'t`；
# 3. 扫描`know` → 匹配`[\p{L}]+` → 提取`know`；
# 4. 扫描`.` → 匹配`[^\s\p{L}\p{N}]+?` → 提取`.`；
# 5. 扫描`你好` → 匹配`[\p{L}]+`（中文属于`\p{L}`）→ 提取`你好`；
# 6. 扫描`123` → 匹配`[\p{N}]+` → 提取`123`；
# 7. 扫描`!` → 匹配`[^\s\p{L}\p{N}]+?` → 提取`!`；

['I', 'don', "'t", 'know', '.', '你好', '123', '!']


#### 2.2.4 构建词频统计结构
预分词后我们得到了`['I', 'don', "'t", '你好', ...]`这样的文本单元列表，但这些是**字符串**，无法直接统计“字节对”（比如`d`和`o`的组合）。所以需要：
1. 把每个文本单元拆成**UTF-8字节**（比如`Hi`→`b'H'`+`b'i'`）；
2. 统计每个“字节序列”的出现次数（比如`Hi`出现了100次）；
3. 把字节序列转成**可修改的结构**（列表），因为后续合并字节对时需要改动（比如把`(b'H', b'i')`合并成`(b'Hi',)`）。

In [24]:
from collections import Counter
raw_counts = Counter()
text = "I don't know. Hi Hi 你好"
words = gpt2_pattern.findall(text)
print(f"原始单词列表：{words}")

# 构建词频统计结构
for word in words:
    byte_tuple = tuple(bytes([b]) for b in word.encode('utf-8'))
    # tuple(bytes([b]) for b in ...)把字节串拆成“单个字节的元组”（比如`b'Hi'`→`(b'H', b'i')`，`b'\xe4\xbd\xa0'`→`(b'\xe4', b'\xbd', b'\xa0')`）
    # 因为`Counter`的键必须是不可变类型（列表是可变的，不能当键），tuple是不可变的，适合做计数的键
    raw_counts[byte_tuple] += 1

# 转成可修改的列表结构
words_list = []
counts_list = []
for word_tuple, freq in raw_counts.items():
    words_list.append(list(word_tuple))
    counts_list.append(freq)

print(f"原始统计（元组+频次）：{raw_counts}")
print(f"可修改的字节列表：{words_list}")
print(f"可修改的频次列表：{counts_list}")

原始单词列表：['I', 'don', "'t", 'know', '.', 'Hi', 'Hi', '你好']
原始统计（元组+频次）：Counter({(b'H', b'i'): 2, (b'I',): 1, (b'd', b'o', b'n'): 1, (b"'", b't'): 1, (b'k', b'n', b'o', b'w'): 1, (b'.',): 1, (b'\xe4', b'\xbd', b'\xa0', b'\xe5', b'\xa5', b'\xbd'): 1})
可修改的字节列表：[[b'I'], [b'd', b'o', b'n'], [b"'", b't'], [b'k', b'n', b'o', b'w'], [b'.'], [b'H', b'i'], [b'\xe4', b'\xbd', b'\xa0', b'\xe5', b'\xa5', b'\xbd']]
可修改的频次列表：[1, 1, 1, 1, 1, 2, 1]


上面的Counter可能不太方便看
```
Counter({
    (b'I',): 1,
    (b'd', b'o', b'n'): 1,
    (b"'", b't'): 1,
    (b'H', b'i'): 2,  # Hi出现2次，频次为2
    (b'\xe4', b'\xbd', b'\xa0', b'\xe5', b'\xa5', b'\xbd'): 1  # 你好的字节元组
})
```
#### 2.2.5 频次状态与倒排索引
每次想合并“最频繁的字节对”，需要：
1. 遍历**所有单词**的字节序列，统计所有字节对的频次；
2. 找到频次最高的字节对；
3. 再遍历**所有单词**，把包含这个字节对的地方合并。

如果有10万个单词，每次合并都要遍历10万次，合并几千次后，总计算量会爆炸。

因此我们引入`states`（频次状态）和`indices`（倒排索引）：
- `states`：提前统计好所有字节对的总频次，不用每次重新算；
- `indices`：记录“每个字节对出现在哪些单词里”，合并时只改这些单词，不用遍历全部。

In [25]:
from collections import defaultdict

states = defaultdict(int)     # int 表示默认值是 0（因为 int() 调用返回 0）。
indices = defaultdict(set)    # set 表示默认值是「空集合」（因为 set() 调用返回 set()）

for idx, word in enumerate(words_list): # 
    freq = counts_list[idx]              # 这个单词在语料中出现的总次数（比如hi出现2次，freq=2）
    # 遍历单词内的所有相邻字节对
    for i in range(len(word) - 1):
        pair = (word[i], word[i+1])      # 比如low的字节列表[b'l',b'o',b'w']，会生成(l,o)、(o,w)
        states[pair] += freq
        indices[pair].add(idx)           # 把当前单词的下标加入该字节对的集合（比如hi出现2次，indices[(h,i)]={0,1}）

print(f"states（字节对+总频次）：{states}")
print(f"indices（字节对+单词索引下标）：{indices}")


states（字节对+总频次）：defaultdict(<class 'int'>, {(b'd', b'o'): 1, (b'o', b'n'): 1, (b"'", b't'): 1, (b'k', b'n'): 1, (b'n', b'o'): 1, (b'o', b'w'): 1, (b'H', b'i'): 2, (b'\xe4', b'\xbd'): 1, (b'\xbd', b'\xa0'): 1, (b'\xa0', b'\xe5'): 1, (b'\xe5', b'\xa5'): 1, (b'\xa5', b'\xbd'): 1})
indices（字节对+单词索引下标）：defaultdict(<class 'set'>, {(b'd', b'o'): {1}, (b'o', b'n'): {1}, (b"'", b't'): {2}, (b'k', b'n'): {3}, (b'n', b'o'): {3}, (b'o', b'w'): {3}, (b'H', b'i'): {5}, (b'\xe4', b'\xbd'): {6}, (b'\xbd', b'\xa0'): {6}, (b'\xa0', b'\xe5'): {6}, (b'\xe5', b'\xa5'): {6}, (b'\xa5', b'\xbd'): {6}})


> 普通`dict={}`如果访问一个不存在的键，会直接报错`KeyError`。
> 
> 「第一次遇到某个字节对时，自动给它赋初始值，之后再累加 / 添加」，普通 dict 要写额外的判断；而 defaultdict 是「带默认值的字典」，初始化时指定「默认值类型」，当访问不存在的键时，会自动创建这个键，并赋予默认值，不用写额外的判断代码

1. `states`：全局字节对频次统计
   - 作用：一次性统计所有字节对的**总出现频次**，后续找“最频繁的字节对”时，直接查`states`即可，不用重新遍历所有单词；
   - 关键细节：`states[pair] += freq` 而不是`+=1`——因为`words_list`里的每个单词代表“一类重复的单词”（比如hi出现2次，freq=2），所以字节对的频次要按单词的实际出现次数累加，而非单词的个数。

2. `indices`：倒排索引（字节对→单词下标）
   - 作用：记录“每个字节对出现在哪些单词里”，后续合并这个字节对时，**只需要修改这些下标对应的单词**，不用遍历所有单词；
   - 比如要合并`(b'l', b'o')`，只需找到下标0（low）和1（lower）的单词，修改它们的字节列表即可，不用管下标2（hi）的单词；
   - 用`set`存储下标：避免重复（比如同一个单词里的字节对不会重复记录下标），且查询/添加效率高。


| 操作 | 无优化 | 有优化（states+indices） |
|------|--------|--------------------------|
| 找最频繁字节对 | 遍历所有单词，重新统计所有字节对的频次 → 慢 | 直接查states，取值最大的键 → 快 |
| 合并字节对 | 遍历所有单词，检查是否包含该字节对 → 慢 | 查indices，只修改对应下标的单词 → 快 |

这两个结构是BPE算法的核心性能优化，尤其在处理大规模语料时，能把时间复杂度从O(n²)降到接近O(n)，让合并过程可行。

#### 2.2.6 合并字节对

In [26]:
best_pair = max(states.items(), key = lambda x: (x[1], x[0]))[0]
best_pair


(b'H', b'i')

- `states.items()`：取出所有`(字节对, 频次)`的键值对；
- `key=lambda x: (x[1], x[0])`：排序规则——**先按频次降序（x[1]），频次相同按字节对字典序升序（x[0]）**；
- 我们的例子中，`(b'l',b'o')`、`(b'o',b'w')`、`(b'h',b'i')`频次都是2，按字典序，`(b'h',b'i')`会成为`best_pair`（假设字典序：h > l > o）。

In [ ]:
def merge_one_round(states, indices, words_list, counts_list):
    """
    执行一轮BPE（字节对编码）核心合并逻辑
    单轮合并流程：找最佳合并对 → 更新受影响单词 → 清理已合并的字节对统计
    
    参数说明（均为可变对象，函数内修改会直接作用于外部变量）：
        states: dict[(bytes, bytes), int]
            全局字节对频次统计字典，键是相邻字节对，值是该对的总出现频次
        indices: dict[(bytes, bytes), set[int]]
            倒排索引字典，键是相邻字节对，值是包含该对的单词在words_list中的下标集合
        words_list: list[list[bytes]]
            可修改的单词字节列表，每个元素是一个单词拆分成的字节列表（如[b'h', b'i']）
        counts_list: list[int]
            与words_list一一对应，每个元素是对应单词在语料中的总出现频次
    
    返回值：
        best_pair: tuple[bytes, bytes] | None
            本次合并的最佳字节对（频次最高）；若无可用字节对则返回None
    """
    # 1. 前置检查：如果全局字节对统计为空，说明没有可合并的对，直接终止
    # （比如所有单词都只剩单个字节，无相邻对）
    if not states:
        print("无可用的字节对可合并，终止合并")
        return None
    
    # 2. 找到本轮“最佳合并对”——BPE的核心选择逻辑
    # max函数的key规则：
    # - 第一优先级：x[1]（字节对频次）降序 → 优先合并出现次数最多的对
    # - 第二优先级：x[0]（字节对本身）升序 → 频次相同时按字典序排序，保证结果可复现
    # [0]表示只取max返回的(字节对, 频次)中的“字节对”部分
    best_pair = max(states.items(), key=lambda x: (x[1], x[0]))[0]
    
    # 3. 取出所有包含最佳合并对的单词下标，转成列表避免迭代时集合变化（set是动态的）
    # indices[best_pair]存储了所有包含该字节对的单词下标，只处理这些单词→性能优化
    relevant_indices = list(indices[best_pair])
    
    # 遍历每个受影响的单词，执行合并操作
    for idx in relevant_indices:
        # 获取当前单词的字节列表（可修改）和其在语料中的出现频次
        word = words_list[idx]  # 如：[b'h', b'i']
        freq = counts_list[idx] # 如：2（表示该单词在语料中出现2次）
        
        # 遍历单词内的相邻字节对，寻找最佳合并对（用while而非for：合并后单词长度会变）
        # for循环的索引是固定的，合并后单词长度缩短会导致索引错位，while更灵活
        i = 0
        while i < len(word) - 1:  # len(word)-1：保证i+1不越界（只遍历到倒数第二个字节）
            # 找到当前位置匹配最佳合并对的字节
            if word[i] == best_pair[0] and word[i+1] == best_pair[1]:
                # -------------------------- 步骤3.1：扣除旧邻居对的频次 --------------------------
                # 合并后，当前字节对的“左邻居对”和“右邻居对”会消失，需要从全局统计中扣除频次
                # （比如单词是[a, b, c]，合并b+c后，a+b这个对就消失了，要扣减频次）
                # 左邻居对：当前字节的前一个字节 + 当前字节
                if i > 0:  # i>0说明有前一个字节
                    prev_pair = (word[i-1], word[i])
                    states[prev_pair] -= freq  # 扣除该单词的频次（不是减1，因为单词可能出现多次）
                
                # 右邻居对：当前字节的后一个字节 + 后后个字节
                if i < len(word) - 2:  # i+2 < len(word)说明有后后个字节
                    next_pair = (word[i+1], word[i+2])
                    states[next_pair] -= freq  # 扣除频次
                
                # -------------------------- 步骤3.2：执行核心合并操作 --------------------------
                # 生成新Token：将两个字节拼接（如b'h' + b'i' = b'hi'）
                new_token = best_pair[0] + best_pair[1]
                # 替换当前位置的字节为新Token（覆盖第一个字节）
                word[i] = new_token
                # 删除第二个字节（因为两个字节已合并成一个，缩短单词长度）
                del word[i+1]  # 合并后word从[a,b]变成[ab]，长度-1
                
                # -------------------------- 步骤3.3：新增新邻居对的频次 --------------------------
                # 合并后，新Token会和左右邻居形成新的字节对，需要加入全局统计
                # 左新邻居对：前一个字节 + 新Token
                if i > 0:  # 有前一个字节才会形成新对
                    new_prev = (word[i-1], word[i])
                    states[new_prev] += freq  # 累加该单词的频次到新对
                    indices[new_prev].add(idx)  # 记录该单词下标到新对的倒排索引
                
                # 右新邻居对：新Token + 后一个字节（合并后word长度已变，重新判断）
                if i < len(word) - 1:  # 合并后i+1仍在单词范围内
                    new_next = (word[i], word[i+1])
                    states[new_next] += freq  # 累加频次
                    indices[new_next].add(idx)  # 记录下标
            
            # 如果当前位置不匹配最佳对，索引+1继续遍历
            else:
                i += 1
    
    # 4. 清理已合并的字节对——该对已被合并成新Token，不会再出现在任何单词中
    # 从全局统计中删除，避免后续轮次误处理
    if best_pair in states:
        del states[best_pair]  # 删除频次统计
    if best_pair in indices:
        del indices[best_pair] # 删除倒排索引
    
    # 5. 打印本轮合并后的关键状态（保留你原有的输出，方便观察）
    print(f"合并字节对 {best_pair} 后，states 中剩余的字节对频次统计：{states}")
    print(f"合并字节对 {best_pair} 后，indices 中剩余的字节对索引统计：{indices}")
    print(f"合并字节对 {best_pair} 后，words_list 中剩余的单词：{words_list}")
    
    # 返回本次合并的最佳对，用于记录合并规则
    return best_pair

这是比较麻烦的地方，前面的都好理解：
> 先对一串字符串进行预分词，隔离那种特殊Token后通过一种正则去分词得到有语义的列表；
> 然后列表的每一个字节比如don和't被拆成元组，元组转列表和统计频次，这个时候还是元组级别的细粒度；
> 再然后是states和indices，这样一来就得到比元组级别还要更小的字节对级别的细粒度了，然后states这个字典存出现频率,indices这个字典存下标；
> 然后就是一轮BPE合并了

举一个例子: `words_list`只有1个单词：`[b'l', b'o', b'w']`（对应“low”）
- 这个单词的频次`counts_list[0] = 5`（出现5次）
- 初始化后：
  - `states`：`{(b'l',b'o'):5, (b'o',b'w'):5}`（这两个字节对各出现5次）
  - `indices`：`{(b'l',b'o'):{0}, (b'o',b'w'):{0}}`（这两个对都出现在下标0的单词里）
- 第一轮合并选的`best_pair`是`(b'l',b'o')`（假设它是高频对）

##### 步骤1：获取需要更新的单词（通过倒排索引）
```python
relevant_indices = list(indices[best_pair])  # 这里就是 [0]
```
`indices[best_pair]`告诉我们：只有下标0的单词包含`(b'l',b'o')`，所以我们只需要处理这1个单词，不用遍历所有单词（这就是倒排索引的价值）。

##### 步骤2：遍历受影响的单词（这里只有下标0的“low”）
```python
for idx in relevant_indices:  # idx=0
    word = words_list[idx]    # word = [b'l', b'o', b'w']
    freq = counts_list[idx]   # freq=5（这个单词出现5次）
    
    i = 0  # 扫描单词的指针，从第一个字节开始
    while i < len(word) - 1:  # len(word)-1=2，i<2即i=0、1
        # 检查当前位置是否匹配最佳对 (b'l',b'o')
        if word[i] == best_pair[0] and word[i+1] == best_pair[1]:
            # ------------ 关键：匹配到了！开始合并操作 ------------
```

##### 步骤3：先“清理旧邻居”——减少旧字节对的频次
合并`(b'l',b'o')`后，原来和这两个字节相邻的对会消失，所以要把它们的频次减掉（因为这些对不存在了）：
```python
# (1)处理“左邻居”：如果当前位置不是第一个字节，要减左邻居对的频次
# 这里i=0（是第一个字节），所以跳过这一步

# (2)处理“右邻居”：(b'o',b'w') 会因为合并消失，所以频次减5
if i < len(word) - 2:  # i=0 < 1（len(word)-2=1），条件成立
    next_pair = (word[i+1], word[i+2])  # (b'o',b'w')
    states[next_pair] -= freq           # states[(b'o',b'w')] = 5-5=0
    if states[next_pair] == 0:
        del states[next_pair]  # 频次为0，直接删掉这个对
```
这一步做完后，`states`变成：`{(b'l',b'o'):5}`（因为`(b'o',b'w')`被删掉了）。

##### 步骤4：执行合并——修改单词的结构
这是最直观的一步：把`b'l'`和`b'o'`合并成新Token`b'lo'`，并修改单词列表：
```python
new_token = best_pair[0] + best_pair[1]  # b'l' + b'o' = b'lo'
word[i] = new_token                      # word[0] 从 b'l' 改成 b'lo'
del word[i+1]                            # 删除原来的 b'o'（下标1）
```
这一步做完后，原来的`[b'l', b'o', b'w']` → 变成 `[b'lo', b'w']`（单词长度从3变2）。

##### 步骤5：再“添加新邻居”——增加新字节对的频次
合并后生成了新的字节对`(b'lo', b'w')`，需要把它加入`states`和`indices`：
```python
# (1)左邻居：i=0，没有左邻居，跳过
# (2)右邻居：新的对是 (b'lo', b'w')
if i < len(word) - 1:  # 现在word是 [b'lo', b'w']，len-1=1，i=0 < 1，成立
    new_next = (word[i], word[i+1])  # (b'lo', b'w')
    states[new_next] += freq         # states[(b'lo',b'w')] = 0+5=5
    indices[new_next].add(idx)       # indices[(b'lo',b'w')] = {0}
```
这一步做完后，`states`变成：`{(b'l',b'o'):5, (b'lo',b'w'):5}`；`indices`新增了`(b'lo',b'w'):{0}`。

##### 步骤6：指针不用动，继续扫描（因为单词长度变了）
合并后单词变成`[b'lo', b'w']`，i还在0的位置，下一轮循环会检查`(b'lo', b'w')`，但这里已经没有`(b'l',b'o')`了，所以循环结束。

##### 步骤7：清理已合并的最佳对
合并完成后，`(b'l',b'o')`已经不存在于任何单词中了，所以要从`states`和`indices`里删掉：
```python
if best_pair in states: del states[best_pair]  # 删掉 (b'l',b'o')
if best_pair in indices: del indices[best_pair]
```
最终，这次合并后：
- `words_list[0]`：`[b'lo', b'w']`（“low”被拆成了`lo`+`w`）
- `states`：`{(b'lo',b'w'):5}`（只剩新的字节对）
- `indices`：`{(b'lo',b'w'):{0}}`
- `merges`列表：新增`(b'l',b'o')`（记录这次合并规则）
- 词表：新增ID 256（255+1） → `b'lo'`

#### 2.2.7 构建最终词表
首先要明确一个前提：我们需要在合并过程中**记录所有的合并规则**（即每一轮的`best_pair`），通常用一个列表`merges`来保存（比如`merges = [(b'h',b'i'), (b'l',b'o'), ...]`）

In [29]:
# 设定要执行的合并轮数
merge_rounds = 5
# 初始化列表，记录所有合并规则（用于后续构建词表）
merges = []

# 循环执行多轮合并
for round_num in range(merge_rounds):
    print(f"\n========== 第 {round_num+1} 轮合并 ==========")
    # 调用单轮合并函数
    best_pair = merge_one_round(states, indices, words_list, counts_list)
    # 如果返回None（无可合并对），提前终止循环
    if best_pair is None:
        break
    # 记录本次合并的字节对
    merges.append(best_pair)

# 打印最终收集的合并规则
print(f"\n最终合并规则列表（共 {len(merges)} 条）：{merges}")


========== 第 1 轮合并 ==========
合并字节对 (b'\xe5', b'\xa5') 后，states 中剩余的字节对频次统计：defaultdict(<class 'int'>, {(b'd', b'o'): 1, (b'o', b'n'): 1, (b"'", b't'): 1, (b'k', b'n'): 1, (b'n', b'o'): 1, (b'o', b'w'): 1, (b'\xe4', b'\xbd'): 1, (b'\xbd', b'\xa0'): 1, (b'\xa0', b'\xe5'): 0, (b'\xa5', b'\xbd'): 0, (b'\xa0', b'\xe5\xa5'): 1, (b'\xe5\xa5', b'\xbd'): 1})
合并字节对 (b'\xe5', b'\xa5') 后，indices 中剩余的字节对索引统计：defaultdict(<class 'set'>, {(b'd', b'o'): {1}, (b'o', b'n'): {1}, (b"'", b't'): {2}, (b'k', b'n'): {3}, (b'n', b'o'): {3}, (b'o', b'w'): {3}, (b'\xe4', b'\xbd'): {6}, (b'\xbd', b'\xa0'): {6}, (b'\xa0', b'\xe5'): {6}, (b'\xa5', b'\xbd'): {6}, (b'\xa0', b'\xe5\xa5'): {6}, (b'\xe5\xa5', b'\xbd'): {6}})
合并字节对 (b'\xe5', b'\xa5') 后，words_list 中剩余的单词：[[b'I'], [b'd', b'o', b'n'], [b"'", b't'], [b'k', b'n', b'o', b'w'], [b'.'], [b'Hi'], [b'\xe4', b'\xbd', b'\xa0', b'\xe5\xa5', b'\xbd']]

========== 第 2 轮合并 ==========
合并字节对 (b'\xe5\xa5', b'\xbd') 后，states 中剩余的字节对频次统计：defaultdict(<class 'int'>, {(b'd', 

In [32]:
# 构建最终词表（接在合并循环之后）
# 1. 添加合并生成的Token（ID从256开始）
for pair in merges:
    new_id = len(vocab)
    vocab[new_id] = pair[0] + pair[1]
    print(f"新增Token：ID={new_id}, Token={pair[0] + pair[1]}")

# 2. 添加特殊Token
for s_tok in special_tokens:
    new_id = len(vocab)
    vocab[new_id] = s_tok.encode("utf-8")
    print(f"新增特殊Token：ID={new_id}, Token={s_tok}")

新增Token：ID=256, Token=b'\xe5\xa5'
新增Token：ID=257, Token=b'\xe5\xa5\xbd'
新增Token：ID=258, Token=b'\xe4\xbd'
新增Token：ID=259, Token=b'\xe4\xbd\xa0'
新增Token：ID=260, Token=b'\xe4\xbd\xa0\xe5\xa5\xbd'
新增特殊Token：ID=261, Token=<endoftext>
新增特殊Token：ID=262, Token=[PAD]


### 2.3 BPE 的局限性与改进
#### 2.3.1 局限性
- 贪心算法：局部最优≠全局最优；
- 无语言学感知：可能拆分出无意义的片段；
- 前缀不一致：无法保证前缀相同的词有相同的Token拆分。

#### 2.3.2 改进变体
- WordPiece（BERT用）：基于概率而非频率合并；
- SentencePiece：无需预分词，直接处理原始文本；
- Unigram LM：基于语言模型概率选择合并策略。

### 2.4 总结
1. BPE的核心是**贪婪合并高频相邻字节对**，解决了直接用字节导致的序列过长问题，平衡“词典大小”和“序列长度”；
2. 工业级BPE实现的关键：预分词（保证语义完整性）、倒排索引（提升合并效率）、特殊Token隔离（保证原子性）；
3. 代码核心逻辑：初始化基础词表→语料处理→预分词→构建频次/索引→迭代合并→生成最终词表→保存规则。

## 3. Tokenizer 封装

